<a href="https://colab.research.google.com/github/GuoZhuyin/zhuyinasia/blob/main/DeepLearning/modeltest1_TEAM_855_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 隨機挑選訓練集進行測試判斷模型準確率

In [ ]:
import PIL
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torchvision
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

In [ ]:
class_to_index={'banana': 0, 'bareland': 1, 'building': 2, 'carrot': 3, 'corn': 4,
 'dragonfruit': 5, 'garlic': 6, 'guava': 7, 'mountain': 8, 'peanut': 9, 'pineapple': 10,
 'pumpkin': 11, 'rice': 12, 'sky': 13, 'soybean': 14, 'sugarcane': 15, 'tomato': 16}
index_to_class = {v: k for k, v in class_to_index.items()}

In [ ]:
classes=['banana', 'bareland', 'carrot', 'corn', 'dragonfruit', 'garlic', 'guava', 'peanut', 'pineapple', 'pumpkin', 'rice', 'soybean', 'sugarcane', 'tomato', 'building', 'mountain', 'sky']
num_classes = len(classes)
print(num_classes)

In [ ]:
folders=['banana', 'bareland', 'carrot', 'corn', 'dragonfruit', 'garlic', 'guava', 'peanut', 'pineapple', 'pumpkin', 'rice', 'soybean', 'sugarcane', 'tomato']
num_folders = len(folders)
print(num_folders)

In [ ]:
from torchvision import models
#model = models.inception_v3(pretrained=False)
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=False)
model.aux_logits = False
model.fc =  nn.Linear(model.fc.in_features, num_classes)
model = nn.DataParallel(model) #cuda設定

In [ ]:
model.load_state_dict(torch.load('../model/InV3_TEAM_855_5.0.pth'))

輸入的模型參數

In [ ]:
model.eval()

In [ ]:
model = model.cuda()

In [ ]:
from PIL import Image 
fname = subpath=os.path.join("banana", "160107-3-0035.JPG")
im_banana = Image.open(fname)

In [ ]:
from IPython.display import display
display(im_banana) 

In [ ]:
width, height = im_banana.size   # Get dimensions
print(width, height)

In [ ]:
new_width = 512
new_height = 512

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(), # ToTensor : [0, 255] -> [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [ ]:
class MyImgDataset(Dataset):
    """FarmLand single dataset."""

    def __init__(self, image, label=None, transform=None):
        """
        Args:
            image (Image): singe image.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.image = image
        self.label = label
        self.transform = transform
        self.boxes=list()

        count = 0
        width, height = image.size   # Get dimensions
        new_width = 512
        new_height = 512
        stride = 32
        for startx in range(width//8, width*7//8-new_width, stride):
          for starty in range(height//2, height-new_height, stride):
              left = startx
              top = starty
              right = startx + new_width
              bottom = starty + new_height
              self.boxes.append((left, top, right, bottom))
    def __len__(self):
        return len(self.boxes)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
                # Crop the sub image
        left, top, right, bottom = self.boxes[idx]
        im2 = im.crop((left, top, right, bottom))

        if self.transform:
            sample = self.transform(im2)
        else:
            sample = im2
        return sample

In [ ]:
mydataset = MyImgDataset(im_banana, class_to_index['banana'])

In [ ]:
testset = MyImgDataset(im_banana, class_to_index['banana'], test_transform)
test_dataloader = DataLoader(testset, batch_size=32, shuffle=False)

In [ ]:
def accumulate_blocks(stats, arr):
    for index in arr:
      if index == 2 or index == 8 or index == 13:
           continue
      stats[index] +=1

In [ ]:
classes=['banana', 'bareland', 'carrot', 'corn', 'dragonfruit', 'garlic', 'guava', 'peanut', 'pineapple', 'pumpkin', 'rice', 'soybean', 'sugarcane', 'tomato', 'building', 'mountain', 'sky']
num_classes = len(classes)
print(num_classes)

In [ ]:
def judge_class(im, label, model, transform, batch_size):
    testset = MyImgDataset(im, label, transform)
    test_dataloader = DataLoader(testset, batch_size=batch_size, shuffle=False)
    stats = [0 for i in range(17)] 
    for images in test_dataloader:
        images = images.cuda()
        outputs = model(images)
        _, dets = torch.max(outputs, 1)
        accumulate_blocks(stats, dets)
    return stats

In [ ]:
folders=['banana', 'bareland', 'carrot', 'corn', 'dragonfruit', 'garlic', 'guava', 'peanut', 'pineapple', 'pumpkin', 'rice', 'soybean', 'sugarcane', 'tomato']
print(len(folders))

In [ ]:
import glob
from PIL import Image 
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
batch_size = 64 #GPU調整batch
for f in folders:
    subpath=os.path.join(f, "*")
    files = glob.glob(subpath)
    print(f"Categroy:{f}")
    for fname in files:
        im = Image.open(fname)
        label = class_to_index[f]
        stats=judge_class(im, label, model,  test_transform, batch_size)
        curclass=index_to_class[np.argmax(stats)]
        if curclass == f:
            correct_pred[curclass] += 1
            print(f"Right:{fname}->{curclass}:{stats}")
        else:
            print(f"Wrong:{fname}->{curclass}:{stats}")
        total_pred[curclass] += 1

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    if classname not in folders:
        continue
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')